In [11]:
import pandas as pd
import numpy as np
from statistics import mode

In [12]:
data=pd.read_csv('u.data', sep= '\t', header = None)
df = data.iloc[:, 0:4]
df.columns = ['user_id', 'movie_id', 'rating', 'date_time']
data2 = pd.read_csv('u.item', encoding='latin1', header=None, sep='|')
item = data2.iloc[:, 0:2]
item.columns = ['movie_id', 'title']
ratings = pd.merge(left = item, right = df, how = 'left', on = 'movie_id')
movie = ratings.sort_values(by = ['user_id','date_time'])
#movie

In [13]:
ratings

,movie_id,title,user_id,rating,date_time
0,1,Toy Story (1995),308,4,887736532
1,1,Toy Story (1995),287,5,875334088
2,1,Toy Story (1995),148,4,877019411
3,1,Toy Story (1995),280,4,891700426
4,1,Toy Story (1995),66,3,883601324
...,...,...,...,...,...
99995,1678,Mat' i syn (1997),863,1,889289570
99996,1679,B. Monkey (1998),863,3,889289491
99997,1680,Sliding Doors (1998),863,2,889289570
99998,1681,You So Crazy (1994),896,3,887160722


#### popular next movie

In [14]:
for i in range(len(movie)-1):
    if movie.user_id.iloc[i] == movie.user_id.iloc[i+1]:
        movie.next_movies.iloc[i] = movie.title.iloc[i+1]

AttributeError: 'DataFrame' object has no attribute 'next_movies'

In [15]:
movie[movie.title == 'Monty Python and the Holy Grail (1974)'].next_movies.value_counts()

AttributeError: 'DataFrame' object has no attribute 'next_movies'

#### user based

In [16]:
utility_matrix = ratings.pivot_table(index = 'user_id', columns= 'title', values= 'rating')
corr_matrix = utility_matrix.corr()
zeus = utility_matrix.corr()['Zeus and Roxanne (1997)'].sort_values(ascending= False)

#### most popular next movie solution

In [17]:
def get_next_movies(movie_title):
    movie = ratings.sort_values(by = ['user_id','date_time'])[['user_id','rating','date_time','title']]
    movie['next_movies'] = np.nan

    for i in range(len(movie)-1):
        if movie['user_id'].iloc[i] == movie['user_id'].iloc[i+1]:
            movie['next_movies'].iloc[i] = movie['title'].iloc[i+1]
    return pd.DataFrame(movie[movie.title == movie_title]['next_movies'].value_counts().head(10))
get_next_movies('Contact (1997)')

C:\Users\Jun Kim\AppData\Local\Temp\ipykernel_6120\3735608523.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['next_movies'].iloc[i] = movie['title'].iloc[i+1]


,next_movies
"English Patient, The (1996)",65
Scream (1996),43
Air Force One (1997),35
Chasing Amy (1997),28
"Full Monty, The (1997)",28
Good Will Hunting (1997),17
Liar Liar (1997),17
Gattaca (1997),15
L.A. Confidential (1997),13
Conspiracy Theory (1997),10


#### user-based solution

In [19]:
def userbased_movies(userid, movie):
    utility_matrix = ratings.pivot_table(index = 'user_id', columns= 'title', values= 'rating')
    corr_matrix = utility_matrix.corr()
    similar = corr_matrix[movie].sort_values(ascending = False)
    pop_movies = pop_based()
    final = pop_movies.merge(similar, how = 'left', left_index = True, right_index = True)
    final.rename({movie: 'similarity'}, axis = 1, inplace = True)
    userid -= 1
    watched_movies = utility_matrix.iloc[userid].dropna().index
    final_1 = final.drop(index = watched_movies, errors = 'ignore').sort_values(by = 'similarity', ascending = False)

    return final_1.head()

userbased_movies()

TypeError: userbased_movies() missing 2 required positional arguments: 'userid' and 'movie'